Наивный Байесовский классификатор

"Наивный" - т.к. предполагает, что комбинируемые вероятности независимы.

<!-- P(A|B) = P(B|A * P(A) / P(B) -->
$$P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}$$

Допустим, у нас есть набор документов, принадлежащих разным категориям.

Попробуем определить вероятность того, что какой-то документ принадлежит категории, т.е. вероятность P(категория|документ)

<!-- P(кат|док) = P(док|кат) * P(кат) / P(док) -->
$$P(cat|doc) = \frac{P(doc|cat) \cdot P(cat)}{P(doc)}$$

Оценим вероятность появления документа в данной категории P(док|кат) как произведение вероятностей появления каждого слова в категории:

<!-- P(док|кат) = П(P(слово|кат)) -->
$$P(doc|cat) = \prod_{word}{P(word|cat)}$$

<!-- P(слово|кат) = C(слово, кат) / Σ(С(слово(i), кат)) -->
$$P(word|cat) = \frac{C(word, cat)}{L(cat)}$$

где L - длина категории в словах.

Оценим P(кат) как вероятность попадания случайно выбранного доумента в конкретную категорию.

<!-- P(кат) = C(док, кат) / C(док) -->
$$P(cat) = \frac{C(doc, cat)}{C(doc)}$$

P(док) не зависит от категорий и для всех сравниваемых величин одинакова, поэтому можно её не учитывать. Тогда искомая категория:

$$hypothesis = \argmax_{cat}{\prod_{word}{\left( \frac{ C(word, cat)}{L(cat)} \right)} \cdot \frac{C(doc, cat)}{C(doc)}}$$

1. Не забывайте, что гораздо удобнее хранить вероятности в логарифмическом представлении!

2. Понятие стоп-слов: частотные слова, которые встречаются во всех категориях. Такие слова нужно убрать из текстов.

In [ ]:
import nltk
nltk.download('stopwords')

In [3]:
from nltk.corpus import stopwords
" ".join(stopwords.words("russian"))

'и в во не что он на я с со как а то все она так его но да ты к у же вы за бы по только ее мне было вот от меня еще нет о из ему теперь когда даже ну вдруг ли если уже или ни быть был него до вас нибудь опять уж вам ведь там потом себя ничего ей может они тут где есть надо ней для мы тебя их чем была сам чтоб без будто чего раз тоже себе под будет ж тогда кто этот того потому этого какой совсем ним здесь этом один почти мой тем чтобы нее сейчас были куда зачем всех никогда можно при наконец два об другой хоть после над больше тот через эти нас про всего них какая много разве три эту моя впрочем хорошо свою этой перед иногда лучше чуть том нельзя такой им более всегда конечно всю между'

3. Иногда при делении на джва класса нам нужно иметь возможность сказать "не знаю". Тогда вычислим отношение между вероятностями принадлежности к двум категориям. Если оно меньше какого-то заданного порога, то однозначно принять решение нельзя.

4. Скорее всего, для такой классификации конкретные словоформы нам не очень важны. Поэтому воспользуемся pymorphy2 для определения исходной формы каждого слова.

Задание: аналогично предыдущему, но с использованием наивного Байесовского классификатора. Попробуйте построить систему не с двумя, а с тремя классами (или больше).